In [1]:
!date

Fri Jan 19 10:43:12 EST 2024


In [2]:
pwd

'/mmfs1/data/aglinska/BC-sim/Code'

In [3]:
%%time
print('importing')
import ants
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import shutil

importing


/data/aglinska/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/data/aglinska/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/data/aglinska/anaconda3/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


CPU times: user 1.68 s, sys: 731 ms, total: 2.41 s
Wall time: 2min 43s


In [4]:
def safe_mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [5]:
def sphere(shape, radius, position):
    """Generate an n-dimensional spherical mask."""
    # assume shape and position have the same length and contain ints
    # the units are pixels / voxels (px for short)
    # radius is a int or float in px
    assert len(position) == len(shape)
    n = len(shape)
    semisizes = (radius,) * len(shape)

    # genereate the grid for the support points
    # centered at the position indicated by position
    grid = [slice(-x0, dim - x0) for x0, dim in zip(position, shape)]
    position = np.ogrid[grid]
    # calculate the distance of all points from `position` center
    # scaled by the radius
    arr = np.zeros(shape, dtype=float)
    for x_i, semisize in zip(position, semisizes):
        # this can be generalized for exponent != 2
        # in which case `(x_i / semisize)`
        # would become `np.abs(x_i / semisize)`
        arr += (x_i / semisize) ** 2

    # the inner part of the sphere will have distance below or equal to 1
    return arr <= 1.0

In [6]:
def do_selective_tx(temp,coords = (32,32,50),sz1=10,sz2=2,do_plot=True,tmp_dir='/scratch/bergerar/',wiggle_spheres=False):
    # Takes in a template brain image and applies selective deformations to it. Returns a new image
    # Deformations are parametrized by two spheres
    
    if wiggle_spheres==True:
        ## Add random jiggle to spheres?
        sphere_small = temp.new_image_like(sphere(shape=brain_size, radius=sz1, position=tuple(np.array(coords)+np.random.randint(low=-5,high=5,size=3)))*1.0)
        sphere_big = temp.new_image_like(sphere(shape=brain_size, radius=sz2, position=tuple(np.array(coords)+np.random.randint(low=-5,high=5,size=3)))*1.0)
    else:
        sphere_small = temp.new_image_like(sphere(shape=brain_size, radius=sz1, position=coords)*1.0)
        sphere_big = temp.new_image_like(sphere(shape=brain_size, radius=sz2, position=coords)*1.0)

    if do_plot==True:
        temp.plot_ortho(sphere_small*3+sphere_big*7,overlay_cmap='flag',xyz=coords,flat=True,overlay_alpha=.5,title='Deformation Targets',textfontcolor='black')

    mytx = ants.registration(fixed=sphere_big, moving=sphere_small,mask=sphere_big+sphere_small,type_of_transform = 'SyNOnly',outprefix=tmp_dir) # learns a vector field from small to big sphere
    
    warpedBrain = ants.apply_transforms(fixed=temp, moving=temp,transformlist=mytx['fwdtransforms'][0])

    if do_plot==True:
        temp.plot_ortho_double(warpedBrain,xyz=coords,title='Before and After',title_dy=.22,textfontcolor='black')
        (warpedBrain-temp).plot_ortho(flat=True,cmap='bwr',xyz=coords,title='Difference Map',textfontcolor='black')
            
    return warpedBrain

In [7]:
import numpy as np
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

def fit_rsa(rdm_data,rdm_model):
    # Takes in two similarity matrices (n x n), has to be square, symmetrical, and either 1s or 0s on the diagonal
    # vectorizes them 
    # Calculates a correlation between them
    # returns r value (between -1 and 1)
    
    return np.corrcoef(get_triu(rdm_data),get_triu(rdm_model))[0,1]
        
def make_RDM(inVec,data_scale='ratio',metric='euclidean'):
    # Makes an similarity matrix
    # Takes in a data matrix (n x p), subjects as rows, and abitrary number of features 
    # Whether you give it a 5x2 or a 5x10000 matrix, will give back a 5x5 similarity matrix

    vec = inVec # Takes whatever data you give it 
    vec = (vec - min(vec.flatten())) / (max(vec.flatten())-min(vec.flatten())) # Normalizes between 0 and 1
    
    if np.ndim(inVec)==1: # must be at least 2D
        vec = np.vstack((vec,np.zeros(vec.shape))).transpose() 
        # ^ ok ok let me explain 
        # pdist function below, works on >=2D data
        # this is a hack in case you want to use it on 1D data
        # in case you pass it a 1D vector, it adds a column of zeros
        # which doesnt change the distances, 
                   
    mat = squareform(pdist(vec,metric=metric).transpose()) # Takes in and calcs the distances, turns into a dissimilarity matrix

    if data_scale=='ordinal': # In case the data is ordinal, makes into a "same or different" matrix or ones and zeros
        mat[mat!=0]=1 # Make into zeros and ones
        
    return mat

def get_triu(inMat):
    # Nifty function that takes in a similarity matrix (nxn) and vectorizes it (returns the upper triangle values)
    
    assert np.ndim(inMat)==2, 'not 2 dim, wtf' # Check if some dummy (i.e me) passed a nd array instead of a simmmat
    assert inMat.shape[0]==inMat.shape[1], 'not a square' # make sure it's a square

    n = inMat.shape[0]
    triu_vec = inMat[np.triu_indices(n=n,k=1)] # Takes the upper triangle values
    
    return triu_vec



In [8]:
brain_size = (64,64,64)

In [9]:
temp = ants.image_read('../Misc/Atlas_brain_2mm.nii',reorient=False)
temp = ants.resample_image(image=temp,resample_params=brain_size,use_voxels=True,interp_type=1)
temp = temp.iMath_normalize()
temp.to_filename('../Misc/Atlas_brain_2mm_64iso.nii')

## Generate Training description

In [10]:
ofdir = 'synth-data-01' # Where to save description

In [11]:
generate_training_descr = True # Whether to generate the data recipe or not. If you want to generate brains from a specified file, set to False

if generate_training_descr==True:
    #shutil.rmtree(f'../{ofdir}')
    safe_mkdir(f'../{ofdir}')
    safe_mkdir(f'../{ofdir}/ims')
    #adhd_locs = [(32,48,25),(32,22,37),(32,46,41),(32,20,18),(34,40,29)]
    #adhd_locs = [(32,48,25),(32,46,41),(32,20,18)]
    adhd_locs = [(32,48,25)]
    td_locs = [(32,32,32)]
    
    n = 500
    n_subtypes = 1
    tx_log_td = []
    tx_log_adhd = []
    tx_log_subtype = []
    tx_locs_td = []
    tx_locs_adhd = []
    subtypes = []
    
    for s in np.arange(n):
        sub = f'sub-{s:04d}'
        td_sphere1 = np.random.uniform(low=10.0,high=35,size=1)
        td_sphere_diff = np.random.uniform(low=-5.0,high=5.0,size=1)
        td_sphere2 = td_sphere1+td_sphere_diff
        tx_log_td.append( (td_sphere1,td_sphere2) )
        tx_locs_td.append(td_locs[0])
        
        subtype = np.mod(s,n_subtypes)
        subtypes.append(subtype)
        tx_locs_adhd.append(adhd_locs[subtype]) # For TD
        if subtype==0:
            adhd_sphere1 = np.random.uniform(low=5.0,high=15,size=1)
            adhd_sphere_diff = np.random.uniform(low=-3.0,high=3.0,size=1)    
            adhd_sphere2 = adhd_sphere1+adhd_sphere_diff
        elif subtype==1:
            adhd_sphere1 = np.random.uniform(low=6.0,high=15,size=1)
            adhd_sphere_diff = np.random.uniform(low=-5.0,high=-1.0,size=1)    
            adhd_sphere2 = adhd_sphere1+adhd_sphere_diff
        elif subtype==3:
            adhd_sphere1 = np.random.uniform(low=6.0,high=15,size=1)
            adhd_sphere_diff = np.random.uniform(low=1.0,high=5.0,size=1)    
            adhd_sphere2 = adhd_sphere1+adhd_sphere_diff
        elif subtype==4:
            adhd_sphere1 = np.random.uniform(low=6.0,high=15,size=1)
            adhd_sphere_diff = np.random.uniform(low=-5.0,high=-1.0,size=1)    
            adhd_sphere2 = adhd_sphere1+adhd_sphere_diff
        
        tx_log_adhd.append( (adhd_sphere1,adhd_sphere2) )
        tx_log_subtype.append( int(np.mod(s,2)==0)+1 )

    deform_adhd = np.array(tx_log_adhd)[:,:,0]
    deform_td = np.array(tx_log_td)[:,:,0]
    subtypes = np.array(subtypes)
    
    df = pd.DataFrame()
    df['subID'] = np.hstack( ([f'sub-{s:04d}' for s in range(n)],[f'sub-{s:04d}' for s in range(n)]) )
    df['dx'] = np.hstack( (np.repeat(1,n),np.repeat(0,n)) )
    df['subtype'] = np.hstack((np.array([np.mod(s,n_subtypes)+1 for s in range(n)]),np.zeros(n))).astype(int)
    df['path'] = np.hstack( ([f'../{ofdir}/ims/ADHD-sub-{s:04d}.nii.gz' for s in range(n)] , [f'../{ofdir}/ims/TD-sub-{s:04d}.nii.gz' for s in range(n)]) )
    df['td_sphere_1'] = np.hstack( (deform_td[:,0],deform_td[:,0]) )
    df['td_sphere_2'] = np.hstack( (deform_td[:,1],deform_td[:,1]) )
    df['adhd_sphere_1'] = np.hstack( (deform_adhd[:,0],np.repeat(0,n)) )
    df['adhd_sphere_2'] = np.hstack( (deform_adhd[:,1],np.repeat(0,n)) )
    df['td_tx'] = df['td_sphere_1']-df['td_sphere_2']
    df['adhd_tx'] = df['adhd_sphere_1']-df['adhd_sphere_2']
    df['loc_tx_td'] = [(32,32,32) for i in range(df.shape[0])]
    df['loc_tx_adhd'] = [(np.nan,np.nan,np.nan) for i in range(df.shape[0])]
    df['loc_tx_adhd'].iloc[0:n] = tx_locs_adhd
    df.to_csv(f'../{ofdir}/data_description.csv')
    df

# Load Description

In [12]:
df = pd.read_csv(f'../{ofdir}/data_description.csv')
n = int(df.shape[0]/2)

In [13]:
# Set up an ANTSpy temprary directory (otherwise, soemtimes ANTS get confused and crashes)
tmp_dir = '/scratch/aglinska/' + 'ants_temp_'+str(np.random.randint(low=10000,high=110000))+'/'
safe_mkdir(tmp_dir)
print(tmp_dir)

/scratch/aglinska/ants_temp_67726/


## Generate brains

In [14]:
do_plot = False # Do you want plots?
do_generate = True # Do you want files?

if do_generate==True:
    for s in tqdm( np.random.permutation(np.arange(n)) ): # Loops over all rows in the recipe and generates the files. Loops in randomized order in case you want to run a few of these in paralell
        sub = df['subID'].values[s]
        
        td_sphere1 = df['td_sphere_1'].values[s+n]
        td_sphere2 = df['td_sphere_2'].values[s+n]
        td_sphere_loc = eval(df['loc_tx_td'].values[s+n])
        
        
        adhd_sphere1 = df['adhd_sphere_1'].values[s]
        adhd_sphere2 = df['adhd_sphere_2'].values[s]
        adhd_sphere_loc = eval(df['loc_tx_adhd'].values[s])
        
        ofn_adhd = df['path'].values[s]
        ofn_td = df['path'].values[s+n]
        
        if not (os.path.exists(ofn_adhd) and os.path.exists(ofn_td)): # In case running in paralell, checks if file is already crated. if so - skips it. Part of paralell computing hack. 
            
            TD_brain = do_selective_tx(temp,coords = td_sphere_loc,sz1=td_sphere1,sz2=td_sphere2,do_plot=False,tmp_dir=tmp_dir,wiggle_spheres=False) # Generate synthetic control, by ading shared variation to template
            ADHD_brain = do_selective_tx(TD_brain,coords = adhd_sphere_loc,sz1=adhd_sphere1,sz2=adhd_sphere2,do_plot=False,tmp_dir=tmp_dir,wiggle_spheres=True) # Generate synthetic patient, by ading shared and specific variation to template
            
            TD_brain.to_filename(ofn_td)
            ADHD_brain.to_filename(ofn_adhd)
            
            if do_plot==True:
                temp.plot_ortho(flat=True,title='template')
                TD_brain.plot_ortho(flat=True,title='TD')
                ADHD_brain.plot_ortho(flat=True,title='ADHD')
            
                print((adhd_sphere1,adhd_sphere2,adhd_sphere_loc))
                print((td_sphere1,td_sphere2,td_sphere_loc))

100%|██████████████████████████████████████████████████| 500/500 [32:22<00:00,  3.88s/it]


  1%|          | 10/1000 [01:23<2:15:37,  8.22s/it]

  1%|          | 11/1000 [01:30<2:13:38,  8.11s/it]

  1%|          | 12/1000 [01:38<2:08:28,  7.80s/it]

  1%|▏         | 13/1000 [01:45<2:07:31,  7.75s/it]

...

 86%|████████▌ | 860/1000 [32:58<00:32,  4.31it/s]

 90%|████████▉ | 899/1000 [32:58<00:14,  6.79it/s]

 94%|█████████▍| 938/1000 [32:59<00:06, 10.29it/s]

 98%|█████████▊| 981/1000 [32:59<00:01, 15.69it/s]

100%|██████████| 1000/1000 [32:59<00:00,  1.98s/it]

In [15]:
shutil.rmtree(tmp_dir) # Clean up temporary directory

## Load and Format Data

In [16]:
# Stores the data in a data.npy file
%%time
#n = 1000
data_td = np.array([ants.image_read(f'../{ofdir}/ims/TD-sub-{s:04d}.nii.gz').numpy() for s in range(n)])
data_adhd = np.array([ants.image_read(f'../{ofdir}/ims/ADHD-sub-{s:04d}.nii.gz').numpy() for s in range(n)])
print(data_td.shape)
print(data_adhd.shape)

(500, 64, 64, 64)
(500, 64, 64, 64)
CPU times: user 5.1 s, sys: 897 ms, total: 5.99 s
Wall time: 7.01 s


In [17]:
np.save(file=f'../{ofdir}/data.npy',arr=np.concatenate((data_adhd,data_td),axis=0))